In [1]:
import pynq
import dishsoap
import numpy as np

dish = dishsoap.Overlay()

In [2]:
# generate all network states
all_init_states = np.arange(2**dish.net_size, dtype=np.uint64)

for init in all_init_states:
    # pretty-print the network state
    print('init state:', dish.unpack(init))
    # run each network state for some number of iterations
    results = dish.run_synch_np(init, 5)
    result_array = np.array([dish.unpack(r) for r in results])
    print(result_array)

init state: [0 0 0]
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
init state: [1 0 0]
[[1 0 0]
 [0 1 1]
 [1 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 1]]
init state: [0 1 0]
[[0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 1]
 [1 0 1]
 [0 0 1]]
init state: [1 1 0]
[[1 1 0]
 [0 1 1]
 [1 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 1]]
init state: [0 0 1]
[[0 0 1]
 [1 0 0]
 [0 1 1]
 [1 0 1]
 [0 0 1]
 [1 0 0]]
init state: [1 0 1]
[[1 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 1]
 [1 0 1]
 [0 0 1]]
init state: [0 1 1]
[[0 1 1]
 [1 0 1]
 [0 0 1]
 [1 0 0]
 [0 1 1]
 [1 0 1]]
init state: [1 1 1]
[[1 1 1]
 [0 0 1]
 [1 0 0]
 [0 1 1]
 [1 0 1]
 [0 0 1]]


In [3]:
init_state = dishsoap.list_to_packed([1,1,0])
results = pynq.allocate(shape=(2000,), dtype=np.uint64)
%time dish.run_synch_inplace(init_state, results)
print('number states saved:', len(results))
print('first two results:\r\n', dish.unpack(results[:2]))
print('last two results:\r\n',  dish.unpack(results[-2:]))

CPU times: user 909 µs, sys: 0 ns, total: 909 µs
Wall time: 960 µs
number states saved: 2000
first two results:
 [[1 1 0]
 [0 1 1]]
last two results:
 [[1 0 1]
 [0 0 1]]


In [44]:
%%time 
def benchmark(init_states, iterations = 5000):
    print('---allocating all memory---')
    n_states = len(init_states)
    %time pynq.allocate(shape=(iterations*n_states,), dtype=np.uint64)
    results = pynq.allocate(shape=(iterations,n_states), dtype=np.uint64)
    print('---running DiSH---')
    for s, r in zip(init_states, results):
        %time dish.run_synch_inplace(s, r)
    print('---total time---')
    # show first and last, prove we did it all
    #print(dish.unpack(results[:n_states,:1]))
    #print(dish.unpack(results[:n_states,-1:]))
    
benchmark(all_init_states)

---allocating all memory---
CPU times: user 6.68 ms, sys: 20.4 ms, total: 27.1 ms
Wall time: 41.9 ms
---running DiSH---
CPU times: user 1.03 ms, sys: 121 µs, total: 1.15 ms
Wall time: 1.12 ms
CPU times: user 2.89 ms, sys: 0 ns, total: 2.89 ms
Wall time: 2.67 ms
CPU times: user 3.27 ms, sys: 0 ns, total: 3.27 ms
Wall time: 3.01 ms
CPU times: user 3.8 ms, sys: 0 ns, total: 3.8 ms
Wall time: 3.68 ms
CPU times: user 0 ns, sys: 3.41 ms, total: 3.41 ms
Wall time: 3.12 ms
CPU times: user 0 ns, sys: 3.3 ms, total: 3.3 ms
Wall time: 3.02 ms
CPU times: user 0 ns, sys: 3.4 ms, total: 3.4 ms
Wall time: 3.12 ms
CPU times: user 2.95 ms, sys: 347 µs, total: 3.3 ms
Wall time: 3.03 ms
---total time---
CPU times: user 70.5 ms, sys: 41.6 ms, total: 112 ms
Wall time: 134 ms
